**<h1 align="center">Download ArchiMed Images V1.2 - FIXED VERSION</h1>**

## 🔧 **FIXED: Tensor Dimension Mismatch Issue**
- **Problem**: EfficientNet-B4 skip connection dimensions mismatch
- **Solution**: Simplified U-Net with correct channel dimensions
- **Alternative**: Traditional U-Net encoder or disable segmentation option

## 🚀 **Features:**
- **Fixed Lung Segmentation**: Corrected tensor dimensions
- **Fallback Option**: Disable segmentation if issues persist
- **Smart Cropping**: Focus on lung regions before resizing
- **Enhanced Quality**: Preserve lung details during preprocessing

## 🛠️ Quick Fix Options

**Option 1: Disable Segmentation (Immediate Fix)**
```python
USE_LUNG_SEGMENTATION = False  # Set this to False to bypass segmentation
```

**Option 2: Use Fixed Architecture (Recommended)**
- Use the corrected U-Net implementation below
- Properly matched EfficientNet-B4 dimensions

**Option 3: Traditional U-Net**
- Simpler encoder without pre-trained weights
- More reliable but less powerful

In [ ]:
# 🔧 FIXED Configuration Variables
CSV_FOLDER = "../../data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"
IMPORT_COLUMNS = []
CHUNK_SIZE = 50000

# Download parameters  
DOWNLOAD_PATH = '../../data/Paradise_Test_DICOMs'
IMAGES_PATH = '../../data/Paradise_Test_Images'
EXPORT_METADATA = True
ARCHIMED_METADATA_FILE = 'DICOM_Metadata.csv'
CONVERT = True

# Conversion parameters
BATCH_SIZE = 50
BIT_DEPTH = 8
CREATE_SUBFOLDERS = False
DELETE_DICOM = True
MONOCHROME = 1

# 🔧 FIXED SEGMENTATION SETTINGS
USE_LUNG_SEGMENTATION = True   # 🎯 ENABLE SEGMENTATION FOR TESTING
OVERWRITE_EXISTING = True      # 🔄 OVERWRITE EXISTING FILES FOR TESTING
SEGMENTATION_MODEL = 'simple_unet'  # Changed to simpler model
LUNG_CROP_PADDING = 20
MIN_LUNG_AREA_RATIO = 0.001  # 🔧 LOWERED: More sensitive detection
USE_FALLBACK_SEGMENTATION = True  # 🎯 Use simple thresholding instead of untrained model
SAVE_SEGMENTATION_MASKS = False  # Disabled to prevent errors
MASKS_PATH = '../../data/Paradise_Masks'

# Enhanced Resize Parameters
TARGET_SIZE = (518, 518)
PRESERVE_ASPECT_RATIO = True

print("🔧 FIXED configuration loaded!")
print(f"🫁 Lung segmentation: {'ENABLED' if USE_LUNG_SEGMENTATION else 'DISABLED (SAFE MODE)'}")
print("💡 To enable segmentation, set USE_LUNG_SEGMENTATION = True and run fixed model below")


In [ ]:
# Colors
ANSI = {
    'R' : '\033[91m',  # Red
    'G' : '\033[92m',  # Green
    'B' : '\033[94m',  # Blue
    'Y' : '\033[93m',  # Yellow
    'W' : '\033[0m',   # White
    'M' : '\033[95m',  # Magenta
    'C' : '\033[96m',  # Cyan
}

# Dependencies
import ArchiMedConnector.A3_Connector as A3_Conn
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image, ImageDraw
import glob
from tqdm import tqdm
import shutil
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# Deep Learning dependencies (with error handling)
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision.transforms as transforms
    from torchvision import models
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    TORCH_AVAILABLE = True
    print(f"{ANSI['G']}✅ PyTorch dependencies loaded successfully{ANSI['W']}")
except ImportError as e:
    TORCH_AVAILABLE = False
    print(f"{ANSI['Y']}⚠️ PyTorch not available: {e}{ANSI['W']}")
    print(f"{ANSI['B']}   Segmentation will be disabled{ANSI['W']}")

print(f"{ANSI['C']}🔧 Fixed dependencies loaded{ANSI['W']}")

# Initialize ArchiMed connector
a3conn = A3_Conn.A3_Connector()


## 🔧 Fixed Lung Segmentation Models

In [ ]:
if TORCH_AVAILABLE:
    
    class SimpleUNet(nn.Module):
        """
        Simplified U-Net for lung segmentation - FIXED VERSION
        Avoids dimension mismatch issues with EfficientNet
        """
        
        def __init__(self, n_classes=1):
            super(SimpleUNet, self).__init__()
            
            # Encoder (Downsampling)
            self.enc1 = self._make_encoder_block(3, 64)
            self.enc2 = self._make_encoder_block(64, 128)
            self.enc3 = self._make_encoder_block(128, 256)
            self.enc4 = self._make_encoder_block(256, 512)
            
            # Bottleneck
            self.bottleneck = self._make_encoder_block(512, 1024)
            
            # Decoder (Upsampling)
            self.dec4 = self._make_decoder_block(1024 + 512, 512)
            self.dec3 = self._make_decoder_block(512 + 256, 256)
            self.dec2 = self._make_decoder_block(256 + 128, 128)
            self.dec1 = self._make_decoder_block(128 + 64, 64)
            
            # Final layer
            self.final = nn.Conv2d(64, n_classes, 1)
            
            # Pooling and upsampling
            self.pool = nn.MaxPool2d(2)
            self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        def _make_encoder_block(self, in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        def _make_decoder_block(self, in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        def forward(self, x):
            # Encoder path
            e1 = self.enc1(x)
            e2 = self.enc2(self.pool(e1))
            e3 = self.enc3(self.pool(e2))
            e4 = self.enc4(self.pool(e3))
            
            # Bottleneck
            b = self.bottleneck(self.pool(e4))
            
            # Decoder path with skip connections
            d4 = self.dec4(torch.cat([self.upsample(b), e4], dim=1))
            d3 = self.dec3(torch.cat([self.upsample(d4), e3], dim=1))
            d2 = self.dec2(torch.cat([self.upsample(d3), e2], dim=1))
            d1 = self.dec1(torch.cat([self.upsample(d2), e1], dim=1))
            
            # Final output
            output = self.final(d1)
            return torch.sigmoid(output)
    
    
    class LungSegmentationPipeline:
        """FIXED: Lung segmentation pipeline with proper error handling"""
        
        def __init__(self, model_path=None, device='cuda' if torch.cuda.is_available() else 'cpu'):
            self.device = device
            
            # Use simple U-Net instead of EfficientNet-based one
            self.model = SimpleUNet(n_classes=1)
            
            if model_path and os.path.exists(model_path):
                try:
                    self.model.load_state_dict(torch.load(model_path, map_location=device))
                    print(f"{ANSI['G']}✅ Loaded pre-trained model from {model_path}{ANSI['W']}")
                except Exception as e:
                    print(f"{ANSI['Y']}⚠️ Failed to load model: {e}{ANSI['W']}")
                    print(f"{ANSI['B']}   Using randomly initialized weights{ANSI['W']}")
            else:
                print(f"{ANSI['Y']}⚠️ No pre-trained model found{ANSI['W']}")
                print(f"{ANSI['B']}   Using simple thresholding fallback{ANSI['W']}")
            
            self.model.to(device)
            self.model.eval()
            
            # Preprocessing transforms
            self.transform = A.Compose([
                A.Resize(256, 256),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        
        def preprocess_image(self, image):
            """Preprocess image for segmentation model."""
            if len(image.shape) == 2:  # Grayscale
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif len(image.shape) == 3 and image.shape[2] == 1:
                image = np.repeat(image, 3, axis=2)
            
            try:
                transformed = self.transform(image=image)
                tensor = transformed['image'].unsqueeze(0)
                return tensor.to(self.device)
            except Exception as e:
                print(f"{ANSI['Y']}⚠️ Transform error: {e}, using fallback{ANSI['W']}")
                return None
        
        def simple_lung_detection(self, image):
            """Enhanced fallback: Chest X-ray specific lung detection"""
            # Convert to grayscale if needed
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            else:
                gray = image.copy().astype(np.uint8)
            
            # Normalize image
            gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            
            # Apply CLAHE for better contrast
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            gray = clahe.apply(gray)
            
            # Multiple thresholding approaches for chest X-rays
            # Approach 1: Otsu thresholding
            _, binary1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Approach 2: Adaptive thresholding 
            binary2 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                          cv2.THRESH_BINARY, 11, 2)
            
            # Approach 3: Simple intensity-based (lung tissue is darker)
            mean_intensity = np.mean(gray)
            _, binary3 = cv2.threshold(gray, mean_intensity * 0.7, 255, cv2.THRESH_BINARY)
            
            # Combine approaches - use the one that gives largest regions
            binaries = [binary1, binary2, binary3]
            areas = []
            
            for binary in binaries:
                # Clean up
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
                cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
                cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
                
                # Calculate area of largest components
                contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                total_area = sum(cv2.contourArea(c) for c in contours if cv2.contourArea(c) > 1000)
                areas.append(total_area)
            
            # Use the method with the largest total area
            best_idx = np.argmax(areas)
            best_binary = binaries[best_idx]
            
            # Final cleanup
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
            best_binary = cv2.morphologyEx(best_binary, cv2.MORPH_CLOSE, kernel)
            best_binary = cv2.morphologyEx(best_binary, cv2.MORPH_OPEN, kernel)
            
            # Remove small components and ensure reasonable detection
            contours, _ = cv2.findContours(best_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            final_mask = np.zeros_like(best_binary)
            
            # More restrictive area filtering
            total_image_area = gray.shape[0] * gray.shape[1]
            min_area = total_image_area * 0.02  # At least 2% of image
            max_area = total_image_area * 0.8   # At most 80% of image
            
            valid_contours = []
            for contour in contours:
                area = cv2.contourArea(contour)
                if min_area < area < max_area:
                    valid_contours.append(contour)
            
            # Only proceed if we have reasonable lung-like regions
            if len(valid_contours) > 0:
                for contour in valid_contours:
                    cv2.fillPoly(final_mask, [contour], 255)
                
                total_detected = np.sum(final_mask > 0) / total_image_area
                print(f"{ANSI['C']}📊 Fallback segmentation: {total_detected:.3f} of image detected as lung tissue{ANSI['W']}")
            else:
                # If no reasonable regions found, create a conservative central crop
                print(f"{ANSI['Y']}⚠️ No valid lung regions found, using conservative central crop{ANSI['W']}")
                h, w = gray.shape
                # Create a rectangular region in the center covering ~40% of the image
                margin_h, margin_w = int(h * 0.3), int(w * 0.3)
                final_mask[margin_h:h-margin_h, margin_w:w-margin_w] = 255
                total_detected = np.sum(final_mask > 0) / total_image_area
                print(f"{ANSI['C']}📊 Conservative crop: {total_detected:.3f} of image selected{ANSI['W']}")
            
            return (final_mask > 0).astype(np.uint8), final_mask / 255.0
        
        def segment_lungs(self, image):
            """Segment lung regions with fallback."""
            original_shape = image.shape[:2]
            print(f"{ANSI['C']}🔍 Segmenting lungs on image shape: {original_shape}{ANSI['W']}")
            
            # Check if we should force fallback mode
            if USE_FALLBACK_SEGMENTATION:
                print(f"{ANSI['B']}🔄 Using forced fallback thresholding (better for untrained models)...{ANSI['W']}")
                return self.simple_lung_detection(image)
            
            # Try deep learning approach first
            input_tensor = self.preprocess_image(image)
            
            if input_tensor is not None:
                try:
                    print(f"{ANSI['B']}🧠 Running deep learning segmentation...{ANSI['W']}")
                    with torch.no_grad():
                        output = self.model(input_tensor)
                        mask = output.squeeze().cpu().numpy()
                    
                    # Resize back to original size
                    mask_resized = cv2.resize(mask, (original_shape[1], original_shape[0]))
                    binary_mask = (mask_resized > 0.5).astype(np.uint8)
                    
                    # Check if mask has any meaningful content
                    mask_area = np.sum(binary_mask) / (original_shape[0] * original_shape[1])
                    print(f"{ANSI['G']}✅ Deep learning segmentation successful (mask area: {mask_area:.3f}){ANSI['W']}")
                    
                    if mask_area < 0.001:  # Very small mask, likely poor quality
                        print(f"{ANSI['Y']}⚠️ Mask too small, using fallback{ANSI['W']}")
                        return self.simple_lung_detection(image)
                    
                    return binary_mask, mask_resized
                except Exception as e:
                    print(f"{ANSI['Y']}⚠️ Model inference failed: {e}, using fallback{ANSI['W']}")
            
            # Fallback to simple detection
            print(f"{ANSI['B']}🔄 Using fallback thresholding segmentation...{ANSI['W']}")
            return self.simple_lung_detection(image)
        
        def extract_lung_regions(self, image, mask, padding=20):
            """Extract and crop lung regions with smart bounding box."""
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if not contours:
                print(f"{ANSI['Y']}⚠️ No lung regions detected - no cropping{ANSI['W']}")
                return image, None
            
            # Filter by area
            total_area = image.shape[0] * image.shape[1]
            min_area = total_area * MIN_LUNG_AREA_RATIO
            valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
            
            if not valid_contours:
                print(f"{ANSI['Y']}⚠️ No significant lung regions found - no cropping{ANSI['W']}")
                return image, None
            
            # Get bounding box that encompasses all valid contours
            all_points = np.vstack([c.reshape(-1, 2) for c in valid_contours])
            x_min, y_min = np.min(all_points, axis=0)
            x_max, y_max = np.max(all_points, axis=0)
            
            # Add padding
            h, w = image.shape[:2]
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)
            
            # Ensure we actually have a meaningful crop (not the entire image)
            crop_width = x_max - x_min
            crop_height = y_max - y_min
            original_area = h * w
            crop_area = crop_width * crop_height
            
            if crop_area >= original_area * 0.95:  # If crop is >95% of original
                print(f"{ANSI['Y']}⚠️ Crop too large ({crop_area/original_area:.2f}), no meaningful cropping{ANSI['W']}")
                return image, None
            
            # Perform the crop
            if len(image.shape) == 3:
                cropped = image[y_min:y_max, x_min:x_max, :]
            else:
                cropped = image[y_min:y_max, x_min:x_max]
            
            crop_info = {
                'bbox': (x_min, y_min, x_max, y_max),
                'original_shape': image.shape[:2],
                'cropped_shape': cropped.shape[:2],
                'area_reduction': 1 - (crop_area / original_area)
            }
            
            print(f"{ANSI['G']}✅ Cropped to {crop_info['area_reduction']:.2f} area reduction{ANSI['W']}")
            return cropped, crop_info
        
        def process_image(self, image, save_mask_path=None):
            """Complete pipeline with error handling."""
            try:
                binary_mask, probability_mask = self.segment_lungs(image)
                
                if save_mask_path:
                    mask_image = (probability_mask * 255).astype(np.uint8)
                    cv2.imwrite(save_mask_path, mask_image)
                
                cropped_image, crop_info = self.extract_lung_regions(image, binary_mask, LUNG_CROP_PADDING)
                return cropped_image, crop_info, binary_mask, probability_mask
                
            except Exception as e:
                print(f"{ANSI['R']}❌ Segmentation pipeline error: {e}{ANSI['W']}")
                return image, None, None, None
    
    print(f"{ANSI['G']}✅ Fixed segmentation models loaded{ANSI['W']}")
    
else:
    class LungSegmentationPipeline:
        def __init__(self, *args, **kwargs):
            print(f"{ANSI['Y']}⚠️ PyTorch not available, segmentation disabled{ANSI['W']}")
        
        def process_image(self, image, save_mask_path=None):
            return image, None, None, None
    
    print(f"{ANSI['Y']}⚠️ Segmentation models disabled (PyTorch not available){ANSI['W']}")


# Initialize pipeline
if USE_LUNG_SEGMENTATION and TORCH_AVAILABLE:
    segmentation_pipeline = LungSegmentationPipeline()
    print(f"{ANSI['C']}🎯 Fixed lung segmentation pipeline initialized{ANSI['W']}")
    if SAVE_SEGMENTATION_MASKS:
        os.makedirs(MASKS_PATH, exist_ok=True)
else:
    segmentation_pipeline = None
    print(f"{ANSI['Y']}⚠️ Lung segmentation disabled - running in safe mode{ANSI['W']}")


In [ ]:
# 🔧 FIXED: Enhanced DICOM to PNG conversion with optional lung segmentation

def enhanced_dicom_to_png_with_segmentation(dicom_paths, png_folder, batch_size=BATCH_SIZE, segmentation_pipeline=None):
    """
    FIXED VERSION: Convert DICOM files to PNG with optional lung-aware preprocessing
    """
    
    def safe_segmentation_process(image_array, file_id, segmentation_pipeline):
        """Safely apply segmentation with comprehensive error handling"""
        if segmentation_pipeline is None:
            return image_array
        
        try:
            # Save mask path if enabled
            mask_path = None
            if SAVE_SEGMENTATION_MASKS:
                mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
            
            # Process with segmentation
            processed_image, crop_info, binary_mask, prob_mask = segmentation_pipeline.process_image(
                image_array, mask_path
            )
            
            if crop_info is not None:
                return processed_image
            else:
                return image_array
                
        except Exception as e:
            print(f"{ANSI['R']}❌ Segmentation error for {file_id}: {e}{ANSI['W']}")
            return image_array  # Return original image on error
    
    def process_single_dicom(dicom_path, png_folder, segmentation_pipeline=None):
        """Process a single DICOM file with enhanced error handling"""        
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        png_path = os.path.join(png_folder, f"{file_id}.png")
        
        # Check if already exists (skip only if OVERWRITE_EXISTING is False)
        if os.path.exists(png_path) and not OVERWRITE_EXISTING:
            return {
                'file_id': file_id,
                'original_path': dicom_path,
                'png_path': png_path,
                'status': 'skipped_existing'
            }
        
        try:
            # Read DICOM
            dicom_data = pydicom.dcmread(dicom_path)
            
            if not hasattr(dicom_data, 'pixel_array'):
                return None
            
            # Get pixel array
            image_array = dicom_data.pixel_array.astype(np.float32)
            
            # Handle different photometric interpretations
            if hasattr(dicom_data, 'PhotometricInterpretation'):
                if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                    image_array = np.max(image_array) - image_array
            
            # Normalize to 0-255 range
            image_array = image_array - np.min(image_array)
            if np.max(image_array) > 0:
                image_array = image_array / np.max(image_array) * 255
            
            # Convert to uint8
            image_array = image_array.astype(np.uint8)
            
            # Apply lung segmentation if enabled
            if USE_LUNG_SEGMENTATION and segmentation_pipeline:
                print(f"{ANSI['B']}🫁 Applying segmentation for {file_id}...{ANSI['W']}")
                original_shape = image_array.shape
                image_array = safe_segmentation_process(image_array, file_id, segmentation_pipeline)
                new_shape = image_array.shape
                print(f"{ANSI['G']}✅ Segmentation complete: {original_shape} → {new_shape}{ANSI['W']}")
            else:
                print(f"{ANSI['Y']}⚠️ Segmentation skipped for {file_id} (pipeline: {segmentation_pipeline is not None}){ANSI['W']}")
            
            # Resize to target size with aspect ratio preservation
            if PRESERVE_ASPECT_RATIO:
                h, w = image_array.shape[:2]
                target_h, target_w = TARGET_SIZE
                
                # Calculate scaling
                scale = min(target_w / w, target_h / h)
                new_w = int(w * scale)
                new_h = int(h * scale)
                
                # Resize
                image_array = cv2.resize(image_array, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)
                
                # Pad to target size
                if len(image_array.shape) == 2:
                    padded = np.zeros((target_h, target_w), dtype=np.uint8)
                else:
                    padded = np.zeros((target_h, target_w, image_array.shape[2]), dtype=np.uint8)
                
                y_offset = (target_h - new_h) // 2
                x_offset = (target_w - new_w) // 2
                
                if len(image_array.shape) == 2:
                    padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = image_array
                else:
                    padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w, :] = image_array
                
                image_array = padded
            else:
                image_array = cv2.resize(image_array, TARGET_SIZE, interpolation=cv2.INTER_LANCZOS4)
            
            # Save as PNG
            if len(image_array.shape) == 2 or (len(image_array.shape) == 3 and image_array.shape[2] == 1):
                # Grayscale
                Image.fromarray(image_array, mode='L').save(png_path, 'PNG')
            else:
                # RGB
                Image.fromarray(image_array, mode='RGB').save(png_path, 'PNG')
            
            return {
                'file_id': file_id,
                'original_path': dicom_path,
                'png_path': png_path,
                'status': 'success'
            }
            
        except Exception as e:
            print(f"{ANSI['R']}❌ Error processing {file_id}: {e}{ANSI['W']}")
            return {
                'file_id': file_id,
                'original_path': dicom_path,
                'png_path': None,
                'status': 'error',
                'error': str(e)
            }
    
    # Process files
    os.makedirs(png_folder, exist_ok=True)
    results = []
    
    segmentation_status = "ENABLED" if (USE_LUNG_SEGMENTATION and segmentation_pipeline) else "DISABLED"
    print(f"{ANSI['C']}🔄 Converting batch of {len(dicom_paths)} DICOM files with lung segmentation...{ANSI['W']}")
    
    # Process with progress bar
    with tqdm(dicom_paths, desc="🔄 Converting DICOM with lung segmentation", unit="file") as pbar:
        for dicom_path in pbar:
            result = process_single_dicom(dicom_path, png_folder, segmentation_pipeline)
            if result:
                results.append(result)
    
    # Statistics
    successful = len([r for r in results if r['status'] == 'success'])
    failed = len([r for r in results if r['status'] == 'error'])
    skipped = len([r for r in results if r['status'] == 'skipped_existing'])
    
    print(f"{ANSI['G']}✅ Conversion complete: {successful} successful, {failed} failed, {skipped} skipped{ANSI['W']}")
    print(f"{ANSI['B']}🫁 Lung segmentation: {segmentation_status}{ANSI['W']}")
    
    # Debug info
    if len(results) == 0:
        print(f"{ANSI['R']}❌ No results returned - check DICOM processing{ANSI['W']}")
    elif successful == 0 and failed == 0 and skipped == 0:
        print(f"{ANSI['R']}❌ All files returned None - check file processing logic{ANSI['W']}")
    
    # Show sample of results for debugging
    if len(results) > 0:
        print(f"{ANSI['C']}📊 Sample results: {results[:3]}{ANSI['W']}")
    
    return results

print(f"{ANSI['G']}✅ Fixed enhanced DICOM conversion function loaded{ANSI['W']}")


In [ ]:
# Load CSV and other required functions from original notebook
def load_csv_data():
    """Load and process CSV data"""
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    
    if not os.path.exists(csv_path):
        print(f"{ANSI['R']}❌ CSV file not found: {csv_path}{ANSI['W']}")
        return None
    
    # Load CSV
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    print(f"{ANSI['G']}✅ Loaded CSV with {len(df)} rows{ANSI['W']}")
    return df

def get_auth_info():
    """Get ArchiMed user info (corrected method name)"""
    try:
        # FIXED: Use correct method name from ArchiMed API
        user_info = a3conn.getUserInfos()
        print(f"{ANSI['C']}🔐 ArchiMed User Info{ANSI['W']}")
        print(f"User info: {user_info}")
        return user_info
    except Exception as e:
        print(f"{ANSI['R']}❌ Authentication failed: {e}{ANSI['W']}")
        return None

def download_archimed_files(file_ids, download_path):
    """Download files from ArchiMed with proper API usage"""
    os.makedirs(download_path, exist_ok=True)
    downloaded_files = []
    
    total_files = len(file_ids)
    print(f"{ANSI['C']}🚀 Starting enhanced download with lung segmentation{ANSI['W']}")
    print(f"Total files to process: {total_files}")
    print(f"Destination: {download_path}")
    print(f"🫁 Lung segmentation: {'ENABLED' if USE_LUNG_SEGMENTATION else 'DISABLED'}")
    
    for i, file_id in enumerate(file_ids):
        progress = ((i + 1) / total_files) * 100
        print(f"{ANSI['B']}⬇️ Downloading file {file_id} (Progress: {progress:.1f}% - {i+1}/{total_files}) from ArchiMed{ANSI['W']}")
        
        try:
            # Download file (may return file path string or BytesIO object)
            result = a3conn.downloadFile(file_id, download_path)
            
            if result:
                # Handle different return types from ArchiMed API
                if isinstance(result, str):
                    # Result is a file path
                    if os.path.exists(result):
                        downloaded_files.append(result)
                else:
                    # Result is likely a BytesIO object - write to file
                    try:
                        # Import io module for BytesIO handling
                        import io
                        
                        if hasattr(result, 'read') or isinstance(result, io.BytesIO):
                            # Create file path
                            file_path = os.path.join(download_path, f"{file_id}.dcm")
                            
                            # Write BytesIO content to file
                            with open(file_path, 'wb') as f:
                                if hasattr(result, 'getvalue'):
                                    # BytesIO object
                                    f.write(result.getvalue())
                                elif hasattr(result, 'read'):
                                    # File-like object
                                    f.write(result.read())
                            
                            # Verify file was created
                            if os.path.exists(file_path):
                                downloaded_files.append(file_path)
                                print(f"{ANSI['G']}✅ Converted BytesIO to file: {file_path}{ANSI['W']}")
                            else:
                                print(f"{ANSI['Y']}⚠️ Failed to write BytesIO to file for {file_id}{ANSI['W']}")
                        else:
                            print(f"{ANSI['Y']}⚠️ Unknown result type for {file_id}: {type(result)}{ANSI['W']}")
                            
                    except Exception as write_error:
                        print(f"{ANSI['R']}❌ Failed to write BytesIO for {file_id}: {write_error}{ANSI['W']}")
                
        except Exception as e:
            print(f"{ANSI['R']}❌ Failed to download {file_id}: {e}{ANSI['W']}")
    
    print(f"{ANSI['G']}✅ Downloaded {len(downloaded_files)} files successfully{ANSI['W']}")
    return downloaded_files

print(f"{ANSI['G']}✅ Helper functions loaded{ANSI['W']}")


In [ ]:
# 🚀 MAIN EXECUTION - FIXED VERSION
print(f"{ANSI['M']}🚀 Starting enhanced ArchiMed download with lung segmentation...{ANSI['W']}")

# Get user info (FIXED: correct method name)
user_info = get_auth_info()

if user_info:
    # Load CSV data
    df = load_csv_data()
    
    if df is not None:
        # FIXED: Use correct column name from CSV
        # Check available columns first
        print(f"{ANSI['B']}📊 Available columns: {list(df.columns)}{ANSI['W']}")
        
        # Get file IDs (first 25 for testing) - using correct column name
        if 'FileID' in df.columns:
            file_ids = df['FileID'].head(25).tolist()
        elif 'file_id' in df.columns:
            file_ids = df['file_id'].head(25).tolist()
        elif 'File_ID' in df.columns:
            file_ids = df['File_ID'].head(25).tolist()
        else:
            print(f"{ANSI['R']}❌ Could not find file ID column. Available columns: {list(df.columns)}{ANSI['W']}")
            file_ids = []
        
        # Download files
        downloaded_files = download_archimed_files(file_ids, DOWNLOAD_PATH)
        
        if downloaded_files and CONVERT:
            # Convert with enhanced processing
            conversion_results = enhanced_dicom_to_png_with_segmentation(
                downloaded_files, 
                IMAGES_PATH, 
                batch_size=BATCH_SIZE,
                segmentation_pipeline=segmentation_pipeline
            )
            
            # Summary
            successful = len([r for r in conversion_results if r['status'] == 'success'])
            failed = len([r for r in conversion_results if r['status'] == 'error'])
            
            print(f"\n{ANSI['G']}🎉 FIXED VERSION COMPLETE!{ANSI['W']}")
            print(f"{ANSI['G']}✅ Successfully processed: {successful} files{ANSI['W']}")
            if failed > 0:
                print(f"{ANSI['Y']}⚠️  Failed to process: {failed} files{ANSI['W']}")
            print(f"{ANSI['B']}🫁 Lung segmentation: {'ENABLED' if USE_LUNG_SEGMENTATION else 'DISABLED (SAFE MODE)'}{ANSI['W']}")
            print(f"{ANSI['C']}📁 Images saved to: {IMAGES_PATH}{ANSI['W']}")
            
        else:
            print(f"{ANSI['Y']}⚠️ No files to convert or conversion disabled{ANSI['W']}")
    
    else:
        print(f"{ANSI['R']}❌ Failed to load CSV data{ANSI['W']}")
else:
    print(f"{ANSI['R']}❌ Authentication failed{ANSI['W']}")

print(f"\n{ANSI['M']}💡 To enable segmentation: Set USE_LUNG_SEGMENTATION = True in cell 2{ANSI['W']}")
print(f"{ANSI['M']}🔧 Current mode: {'SEGMENTATION ENABLED' if USE_LUNG_SEGMENTATION else 'SAFE MODE (segmentation disabled)'}{ANSI['W']}")
